In [1]:
import time
import re
import base64
import logging
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:


# Configure logging
logging.basicConfig(filename='selenium_image_capture.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set up WebDriver
driver = webdriver.Chrome()

logging.info("WebDriver initiated.")

url = "http://littlenedr4.local/dashboard/robot/mission_control"
driver.get(url)
logging.info(f"URL loaded: {url}")
count = 1; #count of images found
try:
    # Allow time for all scripts and dynamic content to load
    time.sleep(30)
    logging.info("Waited 30 seconds for page and scripts to load.")

    elements = driver.find_elements(By.CSS_SELECTOR, '[data-renderer="SensorMsgs_CompressedImage"]')
    if elements:
        element = elements[0]  # Assuming the first element is the correct one
        style = element.value_of_css_property('background-image')
        logging.info(f"Style attribute fetched: {style}")

        # Use regex to extract URL within 'url("")'
        url_match = re.search(r'url\("([^"]+)"\)', style)
        if url_match:
            image_url = url_match.group(1)
            logging.info(f"Image URL extracted: {image_url}")

            if image_url.startswith('data:image'):
                base64_str = image_url.split('base64,')[1]
                image_data = base64.b64decode(base64_str)
                image = Image.open(BytesIO(image_data))
                image.save(f'image_from_stream_{count}.jpg')
                count = count+1
                logging.info("Image saved successfully.")
            else:
                logging.error("Extracted URL does not contain a base64 image.")
        else:
            logging.error("No URL could be extracted from the style attribute.")

except Exception as e:
    logging.error(f"Error occurred: {e}")

finally:
    driver.quit()
    logging.info("WebDriver session closed.")